<h3>Modeling after filter, interpolate and aggregate.</h3>

<p>Variable used: 
    <ol>
        <li>POD: Auckland</li>
        <li>Size: 40</li>
        <li>Type: HC</li>
    </ol>
</p>

In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
sns.set()
pd.options.display.max_rows = 100

df = pd.read_csv('CR_FC_PREP_0522.csv')

df.head()
df.info()
print(f'Dataset size: {df.shape}')


,POD,RATE
0,2005-07-19,2200.0
1,2005-07-20,2250.0
2,2005-07-21,2300.0
3,2005-07-22,2280.0
4,2005-07-23,2260.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6375 entries, 0 to 6374
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   POD     6375 non-null   object 
 1   RATE    6375 non-null   float64
dtypes: float64(1), object(1)
memory usage: 99.7+ KB
Dataset size: (6375, 2)


<h4>Testing Out Seasonal Arima Model and forecast</h4>

In [7]:
df.set_index('POD', inplace=True)
df.head()
df.tail()

,RATE
POD,
2005-07-19,2200.0
2005-07-20,2250.0
2005-07-21,2300.0
2005-07-22,2280.0
2005-07-23,2260.0


,RATE
POD,
2022-12-27,4121.43
2022-12-28,4142.86
2022-12-29,4164.29
2022-12-30,4185.71
2022-12-31,4207.14


In [8]:
from statsmodels.tsa.arima.model import ARIMA
import pmdarima as pm

# Fit the Auto ARIMA model
model = pm.auto_arima(df['RATE'], seasonal=True, suppress_warnings=True, error_action="ignore", stepwise=True, trace=True, test='adf')
model.summary()

ModuleNotFoundError: No module named 'pmdarima'

<h4>As we are trying to perform a cost rate forecast from shipping vendors. These prices are quoted based on inflation, operating cost and rising petrol cost. Hence, it is rather similar to stocks as these are the factors which also affect stock price. Therefore, it is similar, we can try and use algorithms that are used to forecast stocks based on time.</h4>
<p>There are various machine learning algorithms that can be used to forecast stocks, including:</p>

<ol>
    <li>Linear Regression: This algorithm tries to identify a linear relationship between the historical stock prices and other variables, such as trading volume, market indices, or other economic indicators.</li>
    <li>Decision Trees: Decision trees can be used to create a model that predicts future stock prices based on the current price and other variables such as trading volume, earnings reports, and news sentiment.</li>
    <li>Random Forest: Random Forest is an ensemble method that combines multiple decision trees to improve the accuracy of the stock price prediction model.</li>
    <li>Support Vector Machines (SVM): SVM is a supervised learning algorithm that can be used for classification or regression analysis. In the context of stock price prediction, it can help identify patterns in the historical data and make predictions based on those patterns.</li>
    <li>Neural Networks: Neural networks are a set of algorithms that are designed to recognize patterns. They can be used to predict stock prices based on past performance and other variables.</li>
</ol>











It is worth noting that predicting stock prices is a challenging task and no single algorithm can guarantee accurate predictions. The accuracy of the predictions depends on the quality of the data and the algorithm used. Additionally, stock prices are influenced by many factors, including political events, global economic trends, and company-specific news, which can be difficult to predict accurately.